In [3]:
import math as m
import numpy as np
import pandas as pd
from scipy.stats import norm

import pathlib

def bls_euro_van(S_spot,K_strike,T_term,r_rate,sigma_vol,P_product, call_put):
    S_K = np.log(S_spot/K_strike)
    sqrt_T = np.sqrt(T_term)
    ert = np.exp(-r_rate * T_term)
    sigma_T = sigma_vol * sqrt_T
    sigma_T_S = sigma_T * S_spot
    bls_euro_van_d1 = (S_K + (r_rate + 0.5 * sigma_vol * sigma_vol)
                       * T_term) / (sigma_T)
    bls_euro_van_d2 = bls_euro_van_d1 - sigma_T
    bls_N_d1 = norm.cdf(bls_euro_van_d1)
    bls_N_d2 = norm.cdf(bls_euro_van_d2)
    bls_e_d2 = norm.pdf(bls_euro_van_d2)
    
    call_price = (S_spot * bls_N_d1 - K_strike * ert * bls_N_d2)
    put_price = (K_strike * ert - S_spot + call_price)
    call_delta = bls_N_d1
    put_delta = bls_N_d1 - 1.0
    exotic_price = ert * bls_N_d2
    exotic_delta = (ert * bls_e_d2) / (sigma_T_S)
    if P_product == 'Vanilla European':
        if call_put == 'Call':
            name = 'Vanilla European Option Pricer'
            return (call_price,call_delta,name)
        elif call_put == 'Put':
            name = 'Vanilla European Option Pricer'
            return (put_price, put_delta,name)
    if P_product == 'Binary Option':
        name = 'Binary Option Pricer'
        return (exotic_price, exotic_delta,name)
    
input_dir = pathlib.Path( 'Input Files' )
output_dir = pathlib.Path ('Output Files')

pricing_data = pd.read_excel(input_dir / 'PricingData-1.xls', na_filter = False)

pricing_model= lambda x : bls_euro_van(x['Spot']
                                       ,x['Strike']
                                       ,x['Maturity']
                                       ,x['Rate']
                                       ,x['Volatility']
                                       ,x['Product']
                                       ,x['OptionType'])[2]
price = lambda x : bls_euro_van(x['Spot']
                                       ,x['Strike']
                                       ,x['Maturity']
                                       ,x['Rate']
                                       ,x['Volatility']
                                       ,x['Product']
                                       ,x['OptionType'])[0]

delta = lambda x : bls_euro_van(x['Spot']
                                       ,x['Strike']
                                       ,x['Maturity']
                                       ,x['Rate']
                                       ,x['Volatility']
                                       ,x['Product']
                                       ,x['OptionType'])[1]
pricing_data['Pricing Model'] = pricing_data.apply(pricing_model, axis =1)
pricing_data['Price'] = pricing_data.apply(price, axis =1)
pricing_data['Delta'] = pricing_data.apply(delta, axis =1)

pricing_data.to_excel( output_dir / 'Trade Prices and Deltas.xls', index=False)

pricing_data

,Date,TradeID,Spot,Strike,Maturity,Rate,Volatility,Product,OptionType,Pricing Model,Price,Delta
0,18/05/2019,TradeA,90,95,0.75,0.018,0.26,Vanilla European,Call,Vanilla European Option Pricer,6.504160,0.473059
1,18/05/2019,TradeB,101,106,1.00,0.022,0.33,Vanilla European,Put,Vanilla European Option Pricer,14.802134,-0.466033
2,18/05/2019,TradeC,102,105,0.25,0.013,0.19,Binary Option,Call,Binary Option Pricer,0.373866,0.039008


In [ ]:
price_data = pd.read

